In [14]:
%run "Data_Utilities.ipynb"
%run "Model_Building.ipynb"


2.16.1
2.16.1


Model: "functional_23"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_11      │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1 (Conv2D)      │ (None, 112, 112,  │        864 │ input_layer_11[0… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ bn_Conv1            │ (None, 112, 112,  │        128 │ Conv1[0][0]       │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ Conv1_relu (ReLU)   │ (None, 112, 112,  │          0 │ bn_Conv1[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        288 │ Conv1_relu[0][0]  │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │        128 │ expanded_conv_de… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_dept… │ (None, 112, 112,  │          0 │ expanded_conv_de… │
│ (ReLU)              │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │        512 │ expanded_conv_de… │
│ (Conv2D)            │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ expanded_conv_proj… │ (None, 112, 112,  │         64 │ expanded_conv_pr… │
│ (BatchNormalizatio… │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand      │ (None, 112, 112,  │      1,536 │ expanded_conv_pr… │
│ (Conv2D)            │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_BN   │ (None, 112, 112,  │        384 │ block_1_expand[0… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_expand_relu │ (None, 112, 112,  │          0 │ block_1_expand_B… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_pad         │ (None, 113, 113,  │          0 │ block_1_expand_r… │
│ (ZeroPadding2D)     │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise   │ (None, 56, 56,    │        864 │ block_1_pad[0][0] │
│ (DepthwiseConv2D)   │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │        384 │ block_1_depthwis… │
│ (BatchNormalizatio… │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_depthwise_… │ (None, 56, 56,    │          0 │ block_1_depthwis… │
│ (ReLU)              │ 96)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block_1_project     │ (None, 56, 56,    │      2,304 │ block_1_depthwis

 Total params: 2,266,951 (8.65 MB)

 Trainable params: 8,967 (35.03 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

# Training the Model

In [15]:
import os
os.environ["TF_ENABLE_ONEDNN_OPTS"] = "0"

import tensorflow as tf
print(tf.__version__)


2.16.1


In [16]:
import os
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# Update these paths to where your dataset is extracted on your D: drive.
train_dir = r"D:\Note Classification System\train"
val_dir   = r"D:\Note Classification System\test"  # We use test folder as validation here

batch_size = 16  # Adjust based on your GPU/machine capability
epochs = 20

# Create TensorFlow datasets using functions from Data_Utilities.ipynb
train_dataset = get_dataset(train_dir, batch_size=batch_size, augment=True)
val_dataset   = get_dataset(val_dir, batch_size=batch_size, augment=False)

# Build the model (from Model_Building.ipynb)
model = build_model()

# Set up callbacks for saving the best model and early stopping
# checkpoint = ModelCheckpoint("currency_classifier.h5", monitor="val_loss", save_best_only=True, verbose=1)
checkpoint = ModelCheckpoint("currency_classifier.keras", monitor="val_loss", save_best_only=True, verbose=1)

early_stop = EarlyStopping(monitor="val_loss", patience=5, verbose=1)

# Train the model
history = model.fit(train_dataset,
                    validation_data=val_dataset,
                    epochs=epochs,
                    callbacks=[checkpoint, early_stop])

print("Training complete. Best model saved as 'currency_classifier.h5'.")


Epoch 1/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 442ms/step - accuracy: 0.1618 - loss: 2.2437
Epoch 1: val_loss improved from inf to 2.14846, saving model to currency_classifier.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.1620 - loss: 2.2484 - val_accuracy: 0.1667 - val_loss: 2.1485
Epoch 2/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 426ms/step - accuracy: 0.1473 - loss: 2.1933
Epoch 2: val_loss improved from 2.14846 to 2.04387, saving model to currency_classifier.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 610ms/step - accuracy: 0.1469 - loss: 2.1897 - val_accuracy: 0.1667 - val_loss: 2.0439
Epoch 3/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step - accuracy: 0.1686 - loss: 2.0820
Epoch 3: val_loss improved from 2.04387 to 1.97748, saving model to currency_classifier.keras
10/10 ━━━━━━━━━━━━━━━━━━━━ 6s 612ms/step - accuracy: 0.1682 - loss: 2.0801 - val_accuracy: 0.2381 - val_loss: 1.9775
Epoch 4/20
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 469ms/step - accuracy: 0.2101 - loss: 1.9070
Epoch 4: val_loss improved fr